In [2]:
# This python code will attempt to train on a set of humback whale tail images and then use a test data set to classify 
# the images to identify individual whales in the test data.
# I will attempt to also classify one individual image to see if this individual is the test data

# This is a binary classification problem, trying to predict if an image corresponds to a new individual whale or not.

import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

# Remove user warnings from the output
import warnings
warnings.filterwarnings("ignore", category=UserWarning)

# Define constants
IMAGE_SIZE = (150, 150)
BATCH_SIZE = 32 
EPOCHS = 3
EPOCHS_mid = 50
EPOCHS_high = 100

# Define data directory
data_dir = 'resized/'

# Data preprocessing and augmentation
data_generator = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split=0.2  # This is for splitting data into training and validation sets
)

print("Identify individual whales based on unique patterns on their tails (flukes).")
print("This is an image recognition problem where the neural network learns to distinguish between different whale tails.")
print("This process attempts to use a Convolutional Neural Network (CNN) to accomplish this")

train_generator = data_generator.flow_from_directory(
    data_dir,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
#    class_mode=None,  # No class mode specified
    class_mode='binary',  # Set class_mode to 'binary'
    subset='training'  # Use training data for training
)


validation_generator = data_generator.flow_from_directory(
    data_dir,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
#    class_mode=None,  # No class mode specified
    class_mode='binary',  # Set class_mode to 'binary'
    subset='validation'  # Use validation data for validation
)

# Define CNN model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(512, activation='relu'),
    Dense(1, activation='sigmoid')  # Output layer with 1 neuron for binary classification
])

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Train the model
print()
print("Training the model")
print()
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // BATCH_SIZE,
    epochs=EPOCHS,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // BATCH_SIZE
)

# Save the model
model.save('whale_classification_model.h5')

# Now to test the model with an unknown image
# Load the trained model
model = load_model('whale_classification_model.h5')

# Load and preprocess the unknown image
img_path = 'path_to_your_unknown_image.jpg'  # Replace with the path to your image
img = image.load_img(img_path, target_size=(150, 150))  # Resize to match the input size
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
img_array /= 255.  # Normalize pixel values

# Make predictions
prediction = model.predict(img_array)

# Interpret the results
if prediction < 0.5:
    print("The image belongs to a known individual whale.")
else:
    print("The image belongs to a new individual whale.")



Identify individual whales based on unique patterns on their tails (flukes).
This is an image recognition problem where the neural network learns to distinguish between different whale tails.
This process attempts to use a Convolutional Neural Network (CNN) to accomplish this
Found 6511 images belonging to 2 classes.
Found 1627 images belonging to 2 classes.

Training the model

Epoch 1/3
203/203 ━━━━━━━━━━━━━━━━━━━━ 188s 914ms/step - accuracy: 0.9723 - loss: 0.1574 - val_accuracy: 0.9831 - val_loss: 0.0912
Epoch 2/3
203/203 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9688 - loss: 0.0160 - val_accuracy: 1.0000 - val_loss: 0.0036
Epoch 3/3
203/203 ━━━━━━━━━━━━━━━━━━━━ 214s 1s/step - accuracy: 0.9833 - loss: 0.0859 - val_accuracy: 0.9831 - val_loss: 0.0781
